# Session 11: Advanced Retrieval with LangChain

## Learning Objectives:

- Understand and implement multiple retrieval strategies for RAG
- Compare naive, BM25, multi-query, parent-document, contextual compression, ensemble, and semantic chunking approaches
- Build RAG chains over a health and wellness knowledge base using LangChain and QDrant

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

---

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

> NOTE: Create a `.env` file in this directory with `OPENAI_API_KEY` and `COHERE_API_KEY` to avoid being prompted each time.

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Health and Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, stress management, habits, and common health concerns.

### Data Preparation

We'll load the wellness guide as a single document, then split it into smaller chunks using a `RecursiveCharacterTextSplitter` for our vector store. We also keep the raw (unsplit) document for use with the Parent Document Retriever and Semantic Chunker later.

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/HealthWellnessGuide.txt")
raw_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wellness_docs = text_splitter.split_documents(raw_docs)

Let's verify our data was loaded and split correctly!

In [4]:
print(f"Raw documents: {len(raw_docs)}")
print(f"Split chunks: {len(wellness_docs)}")
print(f"\nExample chunk:\n{wellness_docs[0]}")

Raw documents: 1
Split chunks: 45

Example chunk:
page_content='The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.' metadata={'source': 'data/HealthWellnessGuide.txt'}


## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "wellness_guide".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [5]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = QdrantVectorStore.from_documents(
    wellness_docs,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide",
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [9]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [10]:
naive_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Based on the information provided, exercises that can help with lower back pain include:\n\n1. Cat-Cow Stretch:\n   - Start on your hands and knees.\n   - Alternate between arching your back up (cat) and letting it sag down (cow).\n   - Perform 10-15 repetitions.\n\n2. Bird Dog:\n   - From hands and knees, extend the opposite arm and leg while keeping your core engaged.\n   - Hold each extension for about 5 seconds.\n   - Switch sides and repeat.\n   - Do 10 repetitions per side.\n\n3. Pelvic Tilts:\n   - Lie on your back with knees bent.\n   - Flatten your back against the floor by tightening your abdominal muscles and tilting your pelvis up slightly.\n   - Hold for 10 seconds.\n   - Repeat 8-12 times.\n\nThese exercises are gentle and designed to alleviate lower back discomfort and help prevent future episodes. Always consult with a healthcare professional before starting any new exercise routine, especially if you have chronic or severe lower back pain.'

In [11]:
naive_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical, mental, and cognitive well-being. During sleep, the body repairs tissues, regenerates cells, and releases hormones that regulate growth and appetite. Quality sleep also helps consolidate memories and supports brain function. Additionally, adequate sleep strengthens the immune system, reducing susceptibility to illness, and promotes emotional stability. Chronic lack of sleep or poor sleep quality can negatively impact health, increasing risks for conditions such as heart disease, diabetes, and mental health disorders. Therefore, maintaining good sleep hygiene and ensuring sufficient restful sleep are essential for maintaining overall health.'

In [12]:
naive_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress include practicing deep breathing exercises, progressive muscle relaxation, grounding techniques (such as naming things you see, hear, feel, smell, and taste), taking short walks in nature, and listening to calming music. \n\nFor headaches, natural remedies include staying well-hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark and quiet room, giving gentle massages to the temples and neck, and using essential oils like peppermint or lavender. For some individuals, small amounts of caffeine may help, but it can also hurt, so use it cautiously. Maintaining a regular sleep schedule is also helpful in managing headaches.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [13]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(wellness_docs)

We'll construct the same chain - only changing the retriever.

In [14]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [15]:
bm25_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees. Alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n\n- **Pelvic Tilts:** Lie on your back with knees bent. Flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate lower back discomfort and prevent future episodes.'

In [16]:
bm25_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep is a crucial factor for overall health. Maintaining a consistent sleep schedule and creating an optimal sleep environment—such as keeping the room cool, dark, and quiet—can improve sleep quality. Good sleep hygiene practices, like a relaxing bedtime routine and limiting screen time before bed, support restful sleep. Adequate and quality sleep ensures proper functioning of the immune system, mental health, and physical health. Conversely, poor sleep or insomnia can lead to various health problems, including impaired immunity, mood disturbances, and reduced cognitive function. Therefore, good sleep habits are essential for overall wellness.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing relaxation techniques such as deep breathing exercises and progressive muscle relaxation, engaging in meditation, and incorporating herbal teas like chamomile or valerian root. Additionally, staying well-hydrated, ensuring adequate sleep, managing stress levels, and avoiding common headache triggers (such as dehydration and stress) can help alleviate headaches. It is always good to consult with a healthcare provider before starting any new supplement or technique.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### Answer:

Unlike the naive retriever methods that use cosine similarity for identifying similar embeddings, BM25 is an exact text match retriever. An example where BM25 is better is in identifying error codes in software logs: "Find all instances of IndexError in yesterday's logs." The retriever will specifically search for logs where the exact `IndexError` word (at a token level) shows up. If there was a corpus of additional troubleshooting documents and a dense retriever is used instead, this query may find items having to do with any old error, not just the `IndexError`. 

Moreover, in the embedding space, `KeyError` and `TypeError` are also close to `IndexError`, so a dense retriever may return those options. To combat this, the exact token `IndexError` is retrieved when using BM25.

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [18]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [19]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [20]:
contextual_compression_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n1. **Cat-Cow Stretch**: Start on your hands and knees, then alternate between arching your back up (cat) and letting it sag down (cow). Repeat 10-15 times.\n\n2. **Bird Dog**: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n\n3. **Pelvic Tilts**: Lie on your back with knees bent, tighten your abs, and tilt your pelvis slightly to flatten your lower back against the floor. Hold for 10 seconds and repeat 8-12 times.\n\nAlways consult with a healthcare professional before starting new exercises, especially if you have ongoing back issues.'

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health. It is essential for physical health, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Getting adequate sleep, typically 7-9 hours for adults, helps maintain bodily functions, supports emotional health, and enhances learning and memory. Conversely, poor sleep or sleep disorders like insomnia can negatively impact health, making it important to create a conducive sleep environment and address any sleep issues.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing, progressive muscle relaxation, grounding techniques, taking a short walk in nature, listening to calming music, staying well-hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, gentle massage of the temples and neck, using peppermint or lavender essential oils, and maintaining a regular sleep schedule.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [23]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [24]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
multi_query_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'The exercises that can help with lower back pain include:\n\n1. Cat-Cow Stretch: Start on your hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n\n2. Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Perform 10 repetitions per side.\n\n3. Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten your stomach muscles, and raise your shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n\n4. Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\n5. Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds. Repeat 8-12 times.\n\nThese exercises are recommended to alleviate discomfort and help preven

In [26]:
multi_query_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical well-being, mental health, and cognitive function. During sleep, your body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep (7-9 hours per night) helps boost the immune system, improve mood, and support effective functioning of various bodily systems. Poor or insufficient sleep can lead to health problems such as fatigue, stress, weakened immunity, and increased risk of chronic conditions. Practicing good sleep hygiene and maintaining a consistent sleep routine are important for promoting quality sleep and overall health.'

In [27]:
multi_query_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress include practicing mindfulness and meditation, engaging in regular exercise, maintaining social connections, managing time effectively, setting healthy boundaries, and engaging in hobbies and leisure activities. Immediate stress relief techniques include deep breathing exercises, progressive muscle relaxation, grounding techniques, taking a short walk in nature, and listening to calming music.\n\nFor headaches, natural remedies include staying hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, gently massaging the temples and neck, using essential oils like peppermint or lavender, and maintaining a regular sleep schedule.'

### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### Answer:

In RAG, recall measures the ability of the retriever to find all documents relevant to the user query. Generating multiple versions of the user query creates a wider net that allows the retriever to cover more "surface area" and find more related documents. Sometimes a RAG system might infer a different understanding of the query and incorrectly pull out wrong documents. This reformulation ensures the broadest understanding of the query and thereby finds the broadest set of related documents, thereby increasing recall.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. We split the full document into large "parent" chunks (e.g. 2000 characters).
2. Each parent chunk is further split into smaller "child" chunks (e.g. 400 characters).
3. The child chunks are stored in a VectorStore, while the parent chunks are stored in an in-memory docstore.
4. When we query our Retriever, we do a similarity search comparing our query vector to the child chunks.
5. Instead of returning the child chunks, we return their associated parent chunks.

The basic idea is:

- **Search** for small, focused chunks (better semantic matching)
- **Return** big chunks (richer surrounding context)

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by defining our parent and child splitters.

In [28]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [29]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="wellness_parent_child",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="wellness_parent_child", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [30]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [31]:
parent_document_retriever.add_documents(raw_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [32]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [33]:
parent_document_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'To help alleviate lower back pain, the following exercises are recommended:\n\n1. **Cat-Cow Stretch:** Start on your hands and knees. Arch your back upward (cat) and then sag it down (cow). Repeat 10-15 times.\n2. **Bird Dog:** From hands and knees, extend opposite arm and leg, hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n3. **Partial Crunches:** Lie on your back with knees bent, cross arms over your chest, tighten your stomach muscles, and lift your shoulders off the floor. Do 8-12 repetitions.\n4. **Knee-to-Chest Stretch:** Lie on your back, pull one knee towards your chest, hold for 15-30 seconds, then switch legs.\n5. **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n\nThese exercises should be performed gently, and it’s always best to consult with a healthcare professional before starting any new exercise routine, especi

In [34]:
parent_document_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical health, mental well-being, and cognitive functions. During sleep, your body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Getting the recommended 7-9 hours of quality sleep per night supports these processes and helps maintain a healthy immune system, healthy weight, and emotional stability. Poor sleep or insufficient sleep can lead to fatigue, low energy, headaches, and increased stress levels, which may negatively affect overall health. Therefore, practicing good sleep hygiene and ensuring adequate rest are vital for optimal health.'

In [35]:
parent_document_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing exercises, engaging in progressive muscle relaxation, taking short walks in nature, listening to calming music, applying peppermint or lavender essential oils, and maintaining a regular sleep schedule. Additionally, staying well-hydrated by drinking water can help alleviate headaches caused by dehydration.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [36]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [37]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [38]:
ensemble_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

"Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, then alternate between arching your back up (cat) and letting it sag down (cow). Aim for 10-15 repetitions.\n\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while engaging your core. Hold each extension for about 5 seconds, then switch sides. Do 10 repetitions per side.\n\n- **Pelvic Tilts:** Lie on your back with knees bent, tighten your abdominal muscles, and tilt your pelvis up slightly to flatten your back against the floor. Hold for 10 seconds and repeat 8-12 times.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross your arms over your chest, and gently lift your shoulders off the floor by engaging your stomach muscles. Hold briefly and do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back and pull one knee toward your chest, keeping the other foot flat on the floor. Hold for 15-30 seconds, then switch legs.\n\nThese ex

In [39]:
ensemble_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in maintaining overall health. During sleep, your body engages in important processes such as repairing tissues, consolidating memories, and releasing hormones that regulate growth and appetite. Getting adequate sleep—typically 7-9 hours per night—is essential for both physical and mental well-being. Poor or insufficient sleep can lead to issues like fatigue, weakened immune function, cognitive impairments, and increased risk of chronic conditions. Practicing good sleep hygiene, creating an optimal sleep environment, and managing sleep disorders like insomnia contribute significantly to better health and recovery.'

In [40]:
ensemble_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Deep breathing exercises, such as inhaling for 4 counts, holding for 4, exhaling for 4, and holding again.\n- Progressive muscle relaxation, tensing and releasing muscle groups from toes to head.\n- Grounding techniques, like naming 5 things you see, 4 you hear, 3 you feel, 2 you smell, and 1 you taste.\n- Taking short walks, preferably in nature.\n- Listening to calming music.\n- Applying cold or warm compresses to the head or neck.\n- Resting in a dark, quiet room.\n- Gentle massage of temples and neck.\n- Using essential oils such as peppermint or lavender.\n- Staying well-hydrated by drinking water.\n- Maintaining a regular sleep schedule.\n\nFor managing stress specifically, practices like mindfulness and meditation, engaging in hobbies, social support, and setting healthy boundaries are also beneficial.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [41]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [42]:
semantic_documents = semantic_chunker.split_documents(raw_docs)

Let's create a new vector store.

In [43]:
semantic_vectorstore = QdrantVectorStore.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide_semantic_chunks"
)

We'll use naive retrieval for this example.

In [44]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [45]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [46]:
semantic_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Begin on hands and knees, alternate arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can alleviate discomfort and help prevent future episodes of lower back pain.'

In [47]:
semantic_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is crucial for physical restoration, mental well-being, and cognitive functions. During sleep, the body repairs tissues, consolidates memories, releases hormones that regulate growth and appetite, and performs other essential processes. Adults generally need 7-9 hours of quality sleep per night, which occurs in cycles of about 90 minutes involving REM and non-REM stages. Proper sleep hygiene—such as maintaining a consistent sleep schedule, creating a relaxing bedtime routine, and optimizing the sleep environment—can improve sleep quality. Adequate sleep supports immune function, reduces stress, enhances mood and mental clarity, and lowers the risk of various health issues. Conversely, poor sleep can lead to fatigue, increased stress, weakened immune response, and a higher risk for chronic diseases.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- For stress:\n  - Deep breathing exercises (e.g., inhale for 4 counts, hold, exhale for 4)\n  - Progressive muscle relaxation\n  - Grounding techniques (e.g., identifying objects around you)\n  - Walking outdoors, preferably in nature\n  - Listening to calming music\n  - Practicing mindfulness and meditation\n  \n- For headaches:\n  - Staying hydrated by drinking plenty of water\n  - Applying cold or warm compresses to the head or neck\n  - Resting in a dark, quiet room\n  - Gentle massage of temples and neck\n  - Using calming essential oils like peppermint or lavender\n  - Maintaining a regular sleep schedule\n\nThese approaches focus on natural relaxation, hydration, and sensory techniques to help alleviate stress and headaches.'

### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### Answer:

Semantic chunking splits a corpus based on similar semantic ideas or meanings. The text in an FAQ section of a document, which are usually short and repititive, will likely not be semantically different enough. This means the FAQ section may be one single semantic chunk, which could be undesirable for token efficiency and effective context retrieval. It's also possible that since everything looks very similar, small word choices in the text may increase the "signal to noise ratio", thereby causing too much chunking/splitting to occur. 

If we know ahead of time that FAQs need to be properly chunked, there are a few adjustment we can make to the algorithm:
- Split the FAQs so that each semantic chunk contains a Question and Answer. To do this, we can look for paired Question/Answers, numbering or Q/A headers, or punctuation (like question marks).
- Introduce a minimum chunk size to reduce over-chunking
- Add metadata for different sections within FAQs so that retrieval methods can more easily find the right context


---

# 🤝 Breakout Room Part #2

### 🏗️ Activity #1:

Your task is to evaluate the various Retriever methods against each other.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparison between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

## Create Golden Dataset

In [57]:
from openai import OpenAI
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper, llm_factory
from ragas.embeddings.base import embedding_factory
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer


# openai_client = OpenAI()
# generator_llm = llm_factory(
#     "gpt-4.1-nano",
#     client=openai_client
# )

# # Embeddings for testset generation
# generator_embeddings = embedding_factory(
#     "openai",
#     model="text-embedding-3-small",
#     client=openai_client
# )

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
single_hop_distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm, llm_context=generator.llm_context), 1.0)
]

dataset = generator.generate_with_langchain_docs(
    raw_docs,
    testset_size=10,
    query_distribution=single_hop_distribution,
)

/var/folders/b8/jbm0jh5162z101vvqr99rs9r0000gn/T/ipykernel_5949/918644550.py:22: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
/var/folders/b8/jbm0jh5162z101vvqr99rs9r0000gn/T/ipykernel_5949/918644550.py:23: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())


Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]

...

Applying CosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
# Need to make functions for the following:
# - running ragas evaluation
# - processing ragas results



In [54]:
# Set up evaluation framework
from langchain_core.runnables import Runnable
from ragas import RunConfig, evaluate, EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, FactualCorrectness, ContextEntityRecall, NoiseSensitivity
from ragas.testset import Testset

# evaluator_llm = llm_factory(
#         "gpt-4.1-mini",
#         client=openai_client
#     )
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

def evaluate_retriever(retriever_chain: Runnable, dataset: Testset):

    for test_row in dataset:
        response = retriever_chain.invoke({"question" : test_row.eval_sample.user_input})
        test_row.eval_sample.response = response["response"]
        test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

    evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

    custom_run_config = RunConfig(timeout=360)

    results = evaluate(
        dataset=evaluation_dataset,
        metrics=[LLMContextRecall(), FactualCorrectness(), ContextEntityRecall(), NoiseSensitivity()],
        llm=evaluator_llm,
        run_config=custom_run_config
    )
    return results


/var/folders/b8/jbm0jh5162z101vvqr99rs9r0000gn/T/ipykernel_5949/2775136060.py:5: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall, FactualCorrectness, ContextEntityRecall, NoiseSensitivity
/var/folders/b8/jbm0jh5162z101vvqr99rs9r0000gn/T/ipykernel_5949/2775136060.py:5: DeprecationWarning: Importing FactualCorrectness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import FactualCorrectness
  from ragas.metrics import LLMContextRecall, FactualCorrectness, ContextEntityRecall, NoiseSensitivity
/var/folders/b8/jbm0jh5162z101vvqr99rs9r0000gn/T/ipykernel_5949/2775136060.py:5: DeprecationWarning: Importing ContextEntityRecall from 'ragas.metrics' is deprecated and will be re

In [ ]:
# Evaluate the retrievers
from src.ragas_utils import run_ragas_evaluation, compare_ragas_results

# Create a list of all retriever chains
retriever_chains = {
    "Naive Retrieval" : naive_retrieval_chain,
    "BM25 Retrieval" : bm25_retrieval_chain,
    "Contextual Compression Retrieval" : contextual_compression_retrieval_chain,
    "Multi-Query Retrieval" : multi_query_retrieval_chain,
    "Parent Document Retrieval" : parent_document_retrieval_chain,
    "Ensemble Retrieval" : ensemble_retrieval_chain
}

# Initialize a list to store Ragas metrics
ragas_metrics = []

for retriever_name, retriever_chain in retriever_chains.items():
    ragas_evals = run_ragas_evaluation(retriever_chain, retriever_name, dataset)
    ragas_metrics.append(ragas_evals)

In [64]:
from src.ragas_utils import run_ragas_evaluation, compare_ragas_results
result = run_ragas_evaluation(bm25_retrieval_chain, "BM25 Retrieval", dataset)
result

Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

{'chain_name': 'BM25 Retrieval',
 'results': {'context_recall': 0.2917, 'factual_correctness(mode=f1)': 0.4733, 'context_entity_recall': 0.0755, 'noise_sensitivity(mode=relevant)': 0.0833},
 'eval_df':                                            user_input  \
 0   As a wellness-focused professional seeking pra...   
 1                           wat r neck rolls gud for?   
 2   How can partial crunches be performed correctl...   
 3   Wut are macronutriants and why are they import...   
 4                 wat temp in celsius for sleep room?   
 5   What Celsius temperature should bedroom be for...   
 6   What are the main recommendations from Chapter...   
 7   Wht are the main things in Chaptar 17 about di...   
 8   What are the recommended elements of an effect...   
 9   What is guided meditation and how can it help ...   
 10  What are some practical digital wellness strat...   
 11                wat is progresive musle relaxashun?   
 
                                    retriev

In [130]:
for test_row in dataset:
    response = naive_retrieval_chain.invoke({"question" : test_row.eval_sample.user_input})
    print(response)

{'response': AIMessage(content='Well-being is critically linked to exercise because regular physical activity offers numerous health benefits that enhance overall quality of life. According to the guide, exercise improves brain health, helps manage weight, and reduces the risk of disease. It also strengthens bones and muscles and enhances the ability to perform daily activities. Additionally, engaging in regular exercise can contribute to better mental health, reduce stress, and promote social connections—all important components of overall well-being.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 1377, 'total_tokens': 1463, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_08e19ef6a1', 'id': 'chat

In [52]:
dd = dataset.to_pandas()
dd["user_input"].astype(str).tolist()

['Neck how do I fix it?',
 'How does strength training contribute to overall health and well-being?',
 'What role do minerals play in maintaining overall health and how can they be incorporated into a balanced diet according to the principles of holistic wellness?',
 'What is understanding stress in the context of holistic wellness?',
 'Stress reduction techs?',
 'Sleep is important how can I get better sleep?',
 'Whaat is the significance of Thrusday in the weekly workout routine for building a healthy habit?',
 'What is mondai in the workout routine?',
 'How does eating fatty fish support immune health?',
 'Bell peppers help immune health?']

In [56]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,persona_name,query_style,query_length,synthesizer_name
0,As someone dedicated to holistic wellness and ...,[Recommended exercises for lower back pain inc...,[PART 1: EXERCISE AND MOVEMENT\n\nChapter 1: U...,The Cat-Cow Stretch is a gentle and effective ...,The Cat-Cow Stretch is recommended as a gentle...,Wellness Enthusiast,PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer
1,How can Neck Rolls be incorporated into a well...,[Neck and Shoulder Tension\nDesk work and poor...,[PART 1: EXERCISE AND MOVEMENT\n\nChapter 1: U...,Neck Rolls can be effectively incorporated int...,Neck Rolls can be incorporated into a wellness...,Wellness Enthusiast,PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer
2,wut does Chapter 4 say abot healthy eatin and ...,[Micronutrients:\n- Vitamins: Organic compound...,[PART 2: NUTRITION AND DIET\n\nChapter 4: Fund...,Chapter 4 of the guide talks about the fundame...,Chapter 4: Fundamentals of Healthy Eating expl...,Wellness Enthusiast,MISSPELLED,LONG,single_hop_specific_query_synthesizer
3,wut does chpater 5 say bout meal plannin for w...,[Weekly Meal Planning Steps:\n1. Review your s...,[PART 2: NUTRITION AND DIET\n\nChapter 4: Fund...,Chapter 5: Meal Planning for Wellness emphasiz...,Chapter 5: Meal Planning for Wellness explains...,Wellness Enthusiast,MISSPELLED,LONG,single_hop_specific_query_synthesizer
4,What topics are covered in PART 7 of the welln...,[The Personal Wellness Guide\nA Comprehensive ...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,PART 7 of the wellness guide covers the topic ...,PART 7: LIFESTYLE AND WELLNESS covers Chapter ...,Wellness Enthusiast,PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
5,As a wellness enthusiast seeking to build a ba...,[Sample wellness morning routine:\n- Wake at a...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,"Based on PART 5 of the guide, practical strate...",PART 5 provides a comprehensive approach to bu...,Wellness Enthusiast,PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer
6,How does Chapter 17 say you can support digest...,[Foods that support gut health:\n- Fermented f...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,Chapter 17 emphasizes that supporting digestiv...,Chapter 17 explains that supporting digestive ...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
7,"According to Chapter 14, what are the recommen...",[Sample wellness morning routine:\n- Wake at a...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,"According to Chapter 14, the recommended eleme...",Chapter 14 emphasizes that an intentional morn...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
8,Wht are teh main stratgeis for work-life balnc...,[Signs of poor work-life balance:\n- Constant ...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,The main strategies for work-life balance disc...,Chaptar 19 recomends stratgeis for work-life b...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
9,Wht natral remedees does Chaptar 16 recomend f...,[Chapter 16: Managing Headaches Naturally\n\nH...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,Chapter 16 recommends several natural remedies...,Chapter 16 recommends several natural remedies...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer


In [62]:
print(dataset.samples[0].eval_sample.response)
print(dataset.samples[0].eval_sample.retrieved_contexts)

Partial crunches can be performed correctly as follows: Lie on your back with your knees bent and your feet flat on the floor. Cross your arms over your chest gently. Tighten your abdominal muscles and lift your shoulders off the floor briefly, engaging your core. Then, slowly lower your shoulders back down to the starting position. Aim for 8-12 repetitions.

In a balanced exercise routine, partial crunches play a role in strengthening the core muscles. A strong core provides support to the lower back, which can help reduce pain and improve posture. Incorporating abdominal strengthening exercises like partial crunches alongside stretching and back-specific moves (such as the cat-cow stretch or bird dog) can contribute to overall lower back health and aid in managing or preventing lower back pain. Always ensure proper form and avoid overexertion to prevent strain, especially if you have existing back issues.
['- Partial Crunches: Lie on your back with knees bent, cross arms over chest, 

In [59]:
for test_row in dataset:
    response = naive_retrieval_chain.invoke({"question" : test_row.eval_sample.user_input})
    test_row.eval_sample.response = response["response"].content
    test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [63]:
from ragas.metrics import LLMContextRecall, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

custom_run_config = RunConfig(timeout=360)

baseline_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
baseline_result

/var/folders/b8/jbm0jh5162z101vvqr99rs9r0000gn/T/ipykernel_5949/1753574371.py:1: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
/var/folders/b8/jbm0jh5162z101vvqr99rs9r0000gn/T/ipykernel_5949/1753574371.py:1: DeprecationWarning: Importing ResponseRelevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ResponseRelevancy
  from ragas.metrics import LLMContextRecall, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
/var/folders/b8/jbm0jh5162z101vvqr99rs9r0000gn/T/ipykernel_5949/1753574371.py:1: DeprecationWarning: Importing ContextEntityRecall from 'ragas.metrics' is deprecated and will be remove

Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


{'context_recall': 1.0000, 'answer_relevancy': 0.9522, 'context_entity_recall': 0.5232, 'noise_sensitivity(mode=relevant)': 0.1060}